<a href="https://colab.research.google.com/github/Tahsinoshin/Urban_planning_GAN-SPL3/blob/main/DatasetAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pydrive google-api-python-client google-auth-httplib2 google-auth-oauthlib

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
import zipfile
from PIL import Image


In [ ]:
gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

# Replace 'Your Folder Name' with the name of the folder in Google Drive
folder_name = 'Generated Images'

# Find the folder ID of the folder in Google Drive
file_list = drive.ListFile({'q': f"'root' in parents and trashed=false and title='{folder_name}'"}).GetList()
if len(file_list) == 0:
    print(f"Folder '{folder_name}' not found in Google Drive.")
    exit(1)

folder_id = file_list[0]['id']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import zipfile

# Replace 'Your Parent Folder Name' with the name of the parent folder in your Google Drive
parent_folder_name = 'SortedImages/GeneratedImages'

# Set the parent folder path
parent_folder_path = f'/content/drive/MyDrive/{parent_folder_name}'

# Create a zip file for the renamed images
zip_filename = '/content/renamed_images.zip'

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(parent_folder_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
                ext = os.path.splitext(file)[-1]
                new_name = os.path.join(root, f'{os.path.basename(root)}_{file}')
                os.rename(os.path.join(root, file), os.path.join(root, new_name))
                zipf.write(os.path.join(root, new_name), os.path.relpath(os.path.join(root, new_name), parent_folder_path))

print(f'Zipped files to {zip_filename}')


Zipped files to /content/renamed_images.zip


In [ ]:
from google.colab import files

files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# dataset augmentation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
data_dir = '/content/drive/MyDrive/FilteredAndCombined'


In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the directory containing your images
data_dir = '/content/drive/MyDrive/FilteredAndCombined'

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create a directory to save augmented images
output_directory = '/content/augmented_images'
os.makedirs(output_directory, exist_ok=True)

# Generate augmented images and save them
for filename in os.listdir(data_dir):
    image_path = os.path.join(data_dir, filename)
    image = Image.open(image_path)
    image = np.array(image)
    image = np.expand_dims(image, axis=0)

    i = 0
    for batch in datagen.flow(image, batch_size=1):
        augmented_image = batch[0].astype('uint8')
        augmented_image = Image.fromarray(augmented_image)
        augmented_image.save(os.path.join(output_directory, f'augmented_{i}_{filename}'))
        i += 1
        if i >= 3:  # Create three augmented versions for each image
            break


In [ ]:
import shutil

shutil.make_archive('/content/generated_images', 'zip', output_directory)


'/content/generated_images.zip'

In [ ]:
from google.colab import files

files.download('/content/generated_images.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>